Connect to the database
----------------------------------------

In [1]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
#!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
#!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
#!pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
# Enter the connection string for your Db2 on Cloud database instance below

%sql ibm_db_sa://prj63437:HHrZHS29hKohRSpb@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb?security=SSL

Problem 1
------------------
Find the total number of crimes recorded in the CRIME table.

In [4]:
%sql select count(CASE_NUMBER) as total_number_crimes from CHICAGO_CRIME_DATA

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


total_number_crimes
533


Problem 2
---------------
List community areas with per capita income less than 11000.

In [5]:
%sql select COMMUNITY_AREA_NAME, PER_CAPITA_INCOME from CENSUS_DATA where PER_CAPITA_INCOME < 11000

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


community_area_name,per_capita_income
West Garfield Park,10934
South Lawndale,10402
Fuller Park,10432
Riverdale,8201


Problem 3
-----------
List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [6]:
%sql select CASE_NUMBER from CHICAGO_CRIME_DATA \
where DESCRIPTION LIKE '%MINOR'

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


case_number
HL266884
HK238408


Problem 4
--------------
List all kidnapping crimes involving a child?

In [7]:
%sql select * from CHICAGO_CRIME_DATA \
where DESCRIPTION LIKE '%MINOR%' and PRIMARY_TYPE LIKE '%KIDNAPPING%'

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location


Problem 5
---------
What kinds of crimes were recorded at schools?

In [8]:
%sql SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


primary_type
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPA
NARCOTICS
PUBLIC PEACE VI


Problem 6
------------
List the average safety score for each type of school.

In [9]:
%%sql SELECT "Elementary, Middle or High School", AVG(safety_score) FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle or High School"

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: SQLNumResultCols failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "Elementary, Middle or High School" is not valid in the context where it is used.  SQLSTATE=42703 SQLCODE=-206
[SQL: SELECT "Elementary, Middle or High School" , AVG(safety_score) FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle or High School"]
(Background on this error at: http://sqlalche.me/e/13/f405)


Problem 7
-------------
List 5 community areas with highest % of households below poverty line

In [10]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA \
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5;

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


community_area_name,percent_households_below_poverty
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


Problem 8
--------------
Which community area is most crime prone?

In [11]:
%sql SELECT COUNT(*), COMMUNITY_AREA_NAME CARN FROM CENSUS_DATA C, CHICAGO_CRIME_DATA CR \
WHERE C.COMMUNITY_AREA_NUMBER=CR.COMMUNITY_AREA_NUMBER \
GROUP BY COMMUNITY_AREA_NAME \
ORDER BY CARN DESC LIMIT 1

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


1,carn
8,Woodlawn


Problem 9
---------
Use a sub-query to find the name of the community area with highest hardship index

In [12]:
%sql SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA \
ORDER BY HARDSHIP_INDEX DESC NULLS LAST LIMIT 1

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


community_area_name,hardship_index
Riverdale,98


Problem 10
-------------
Use a sub-query to determine the Community Area Name with most number of crimes?

In [13]:
%%sql select COMMUNITY_AREA_NAME from CENSUS_DATA where COMMUNITY_AREA_NUMBER=(
select COMMUNITY_AREA_NUMBER from CHICAGO_CRIME_DATA
 where COMMUNITY_AREA_NUMBER GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1)

 * ibm_db_sa://prj63437:***@19af6446-6171-4641-8aba-9dcff8e1b6ff.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:30699/bludb
Done.


community_area_name
Austin
